In [25]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Archivos de entrada
raster_utm20 = r"C:\Users\sergi\Desktop\DCI_UTM20_1M.tif" # Ráster a reproyectar
raster_utm21 = r"C:\Users\sergi\Documents\repos\bathypy\data\raster\Satellite\DCI\492_reference.tif" # Ráster de referencia
raster_reproyectado = "DCI_UTM20_1MM.tif"  # Salida alineada

# Leer el ráster de referencia (UTM 20)
with rasterio.open(raster_utm21) as src_ref:
    ref_crs = src_ref.crs  # Obtener la proyección objetivo

# Leer el ráster en UTM 21
with rasterio.open(raster_utm20) as src:
    transform, width, height = calculate_default_transform(
        src.crs, ref_crs, src.width, src.height, *src.bounds
    )
    
    # Crear metadatos del nuevo ráster
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': ref_crs,
        'transform': src_ref.transform,
        'width': src_ref.width,
        'height': src_ref.height
    })

    # Reproyectar y guardar
    with rasterio.open(raster_reproyectado, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):  # Bandas
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=ref_crs,
                resampling=Resampling.average
            )

print("Reproyección y alineación completadas.")

Reproyección y alineación completadas.


In [28]:
import rasterio
import numpy as np


filenames = [
r"C:\Users\sergi\Documents\repos\gee_acolite\jupyters\DCI_UTM20_1M.tif",
r"C:\Users\sergi\Documents\repos\gee_acolite\jupyters\DCI_UTM20_2M.tif",
r"C:\Users\sergi\Documents\repos\gee_acolite\jupyters\DCI_UTM20_4M.tif",
]

with rasterio.open(filenames[0]) as src:
    data = src.read()
    data[:] = np.nan
    profile = src.profile
    profile['nodata'] = np.nan
    no_values = np.isnan(data)

print(data.shape)

with rasterio.open(r"C:\Users\sergi\Documents\repos\gee_acolite\jupyters\DCI_UTM20_ALL.tif", 'w', **profile) as dst:
    for filename in filenames:
        with rasterio.open(filename) as src:
            depth = src.read()
            depth[depth == -99999] = np.nan
            no_values &= np.isnan(depth)
            data = np.nansum([depth, data], axis = 0)
            data[no_values] = np.nan

    dst.write(data)

data

(1, 2138, 2173)


array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])